In [1]:
xor2 = [
    ([0,0], [1]),
    ([0,1], [0]),
    ([1,0], [0]),
    ([1,1], [1]),
]

xor3 = [
    ([0,0,0], [1]),
    ([0,0,1], [0]),
    ([0,1,0], [0]),
    ([0,1,1], [1]),
    ([1,0,0], [0]),
    ([1,0,1], [1]),
    ([1,1,0], [1]),
    ([1,1,1], [0]),
]

xor4 = [
    ([0,0,0,0], [1]),
    ([0,0,0,1], [0]),
    ([0,0,1,0], [0]),
    ([0,0,1,1], [1]),
    ([0,1,0,0], [0]),
    ([0,1,0,1], [1]),
    ([0,1,1,0], [1]),
    ([0,1,1,1], [0]),
    ([1,0,0,0], [0]),
    ([1,0,0,1], [1]),
    ([1,0,1,0], [0]),
    ([1,0,1,1], [1]),
    ([1,1,0,0], [1]),
    ([1,1,0,1], [0]),
    ([1,1,1,0], [0]),
    ([1,1,1,1], [1]),
]

In [2]:
from src.mlp import MLP
rede = MLP([4,4,1])

In [3]:
for a in range(2):
    for b in range(2):
        for c in range(2):
            for d in range(2):
                y, _ = rede.feedforward([a,b,c,d])
                print('XOR({}, {}, {}, {}) = {}'.format(a,b,c,d,y[0]))

XOR(0, 0, 0, 0) = [0.21463486]
XOR(0, 0, 0, 1) = [0.2100035]
XOR(0, 0, 1, 0) = [0.20900741]
XOR(0, 0, 1, 1) = [0.20499651]
XOR(0, 1, 0, 0) = [0.17371436]
XOR(0, 1, 0, 1) = [0.16790055]
XOR(0, 1, 1, 0) = [0.16886776]
XOR(0, 1, 1, 1) = [0.16424145]
XOR(1, 0, 0, 0) = [0.21858518]
XOR(1, 0, 0, 1) = [0.20437045]
XOR(1, 0, 1, 0) = [0.21547498]
XOR(1, 0, 1, 1) = [0.20253292]
XOR(1, 1, 0, 0) = [0.17654239]
XOR(1, 1, 0, 1) = [0.1707626]
XOR(1, 1, 1, 0) = [0.16917868]
XOR(1, 1, 1, 1) = [0.16568653]


In [4]:
rede.treino(amostras=xor4, lim_epocas=20000, taxa=0.5)

Epoca: 00000 | MSE: 0.32411712
Epoca: 01000 | MSE: 0.00595344
Epoca: 02000 | MSE: 0.00139312
Epoca: 03000 | MSE: 0.00077244
Epoca: 04000 | MSE: 0.00053162
Epoca: 05000 | MSE: 0.00040438
Epoca: 06000 | MSE: 0.00032590
Epoca: 07000 | MSE: 0.00027272
Epoca: 08000 | MSE: 0.00023434
Epoca: 09000 | MSE: 0.00020536
Epoca: 10000 | MSE: 0.00018270
Epoca: 11000 | MSE: 0.00016452
Epoca: 12000 | MSE: 0.00014960
Epoca: 13000 | MSE: 0.00013714
Epoca: 14000 | MSE: 0.00012658
Epoca: 15000 | MSE: 0.00011752
Epoca: 16000 | MSE: 0.00010967
Epoca: 17000 | MSE: 0.00010279
Epoca: 18000 | MSE: 0.00009671
Epoca: 19000 | MSE: 0.00009131


In [5]:
for a in range(2):
    for b in range(2):
        for c in range(2):
            for d in range(2):
                y, _ = rede.feedforward([a,b,c,d])
                print('XOR({}, {}, {}, {}) = {}'.format(a,b,c,d,y[0]))

XOR(0, 0, 0, 0) = [0.99555986]
XOR(0, 0, 0, 1) = [0.01157549]
XOR(0, 0, 1, 0) = [0.00367719]
XOR(0, 0, 1, 1) = [0.99528413]
XOR(0, 1, 0, 0) = [0.0104507]
XOR(0, 1, 0, 1) = [0.98486091]
XOR(0, 1, 1, 0) = [0.98927432]
XOR(0, 1, 1, 1) = [0.01059214]
XOR(1, 0, 0, 0) = [0.01559851]
XOR(1, 0, 0, 1) = [0.98956459]
XOR(1, 0, 1, 0) = [0.00019072]
XOR(1, 0, 1, 1) = [0.99070463]
XOR(1, 1, 0, 0) = [0.99567343]
XOR(1, 1, 0, 1) = [0.01201]
XOR(1, 1, 1, 0) = [0.00295769]
XOR(1, 1, 1, 1) = [0.99577135]


In [6]:
rede.pesos

[array([[-5.95342282, -4.87325104,  6.65450889, -5.64635833],
        [ 5.9168529 ,  4.81206017, -6.42571145,  5.56925839],
        [ 1.28562701,  6.52240846,  6.70733304, -3.08521071],
        [-1.96599279, -6.56759653, -6.56256329,  9.68446272]]),
 array([[ 10.10748732],
        [-10.1296559 ],
        [ 10.42674969],
        [  9.5264129 ]])]

In [7]:
rede.bias

[array([[-0.76306271, -1.98443485,  2.78183839,  2.03719713]]),
 array([[-14.82348556]])]